# Import Library
Melakukan import library yang dibutuhkan sebelum memulai tahapan.

In [1]:
import os
import warnings
warnings.filterwarnings("ignore")
from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen, ExampleValidator, Transform, Trainer, Tuner, Pusher, Evaluator
from tfx.proto import example_gen_pb2, trainer_pb2, pusher_pb2
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.dsl.components.common.resolver import Resolver 
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import LatestBlessedModelStrategy 
from tfx.types import Channel 
from tfx.types.standard_artifacts import Model, ModelBlessing 
import tensorflow_model_analysis as tfma 

# Set Variable
Pertama, definisikan beberapa variabel yang dibutuhkan untuk pipeline ini, seperti nama pipeline, lokasi penyimpanan artifact, path metadata, dan direktori model yang akan dihasilkan.

In [2]:
PIPELINE_NAME = "hoax-detection"
SCHEMA_PIPELINE_NAME = "hoax-tfdv-schema"

#Directory untuk menyimpan artifact yang akan dihasilkan
PIPELINE_ROOT = os.path.join('outputs', PIPELINE_NAME)

# Path to a SQLite DB file to use as an MLMD storage.
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')

# Output directory where created models from the pipeline will be exported.
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)

In [3]:
DATA_ROOT = "data"

In [4]:
interactive_context = InteractiveContext(pipeline_root=PIPELINE_ROOT)

# Data Ingestion
Pada tahap ini, lakukan injeksi data mentah ke dalam pipeline Menggunakan CsvExampleGen. data akan diambil dari direktori yang telah ditentukan dan dibagi menjadi data pelatihan dan evaluasi.

In [5]:
output = example_gen_pb2.Output(
    split_config = example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name="train", hash_buckets=8),
        example_gen_pb2.SplitConfig.Split(name="eval", hash_buckets=2)
    ])
)
example_gen = CsvExampleGen(input_base=DATA_ROOT, output_config=output)

In [6]:
interactive_context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 94
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

# Data Validation

## Summary Statistik
Pada tahap ini, menghasilkan statistik dari data yang telah diingest menggunakan StatisticsGen. Statistik ini memberikan gambaran umum tentang distribusi data.

In [7]:
statistics_gen = StatisticsGen(
    examples=example_gen.outputs["examples"]
)
 
 
interactive_context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 95
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

In [8]:
interactive_context.show(statistics_gen.outputs["statistics"])

## Data Schema
Setelah mendapatkan statistik, menghasilkan skema data menggunakan SchemaGen. Skema ini membantu dalam memahami struktur dan tipe data.

In [9]:
schema_gen = SchemaGen(statistics=statistics_gen.outputs["statistics"])
interactive_context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 96
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

In [10]:
interactive_context.show(schema_gen.outputs["schema"])

,Type,Presence,Valency,Domain
Feature name,,,,
'label',INT,required,,-
'narasi',BYTES,required,,-


## Identification Anomaly
Memeriksa adanya anomali dalam data menggunakan ExampleValidator dengan membandingkan data terhadap skema yang telah dibuat.

In [11]:
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema']
)
interactive_context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 97
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}))

In [12]:
interactive_context.show(example_validator.outputs['anomalies'])

# Data Preprocessing
Melakukan preprocessing pada data menggunakan Transform. Sediakan file modul transformasi yang berisi fungsi-fungsi transformasi.

In [13]:
TRANSFORM_MODULE_FILE = "modules/transform.py"

In [14]:
transform  = Transform(
    examples=example_gen.outputs['examples'],
    schema= schema_gen.outputs['schema'],
    module_file=os.path.abspath(TRANSFORM_MODULE_FILE)
)
interactive_context.run(transform)

Instructions for updating:
Use ref() instead.


Instructions for updating:
Use ref() instead.


INFO:tensorflow:Assets written to: outputs\hoax-detection\Transform\transform_graph\98\.temp_path\tftransform_tmp\24c8898defa2448ea26dbe9e0e40d273\assets


INFO:tensorflow:Assets written to: outputs\hoax-detection\Transform\transform_graph\98\.temp_path\tftransform_tmp\24c8898defa2448ea26dbe9e0e40d273\assets


INFO:tensorflow:struct2tensor is not available.


...

INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 98
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={})
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={})
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={})
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={})
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={})
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}))

# Pengembangan Model

## Tuner
Melakukan tuning hyperparameter menggunakan Tuner. Modul tuner berisi logika untuk mencari hyperparameter terbaik.

In [15]:
TUNER_MODULE_FILE = "modules/tuner.py"

In [16]:
tuner = Tuner(
    module_file=os.path.abspath(TUNER_MODULE_FILE),
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    train_args=trainer_pb2.TrainArgs(splits=['train']),
    eval_args=trainer_pb2.EvalArgs(splits=['eval'])
)

In [17]:
interactive_context.run(tuner)

Trial 9 Complete [00h 00m 21s]
val_loss: 0.4868699312210083

Best val_loss So Far: 0.4690980315208435
Total elapsed time: 00h 02m 21s

Search: Running Trial #10

Value             |Best Value So Far |Hyperparameter
10000             |12000             |vocab_size
150               |75                |sequence_length
120               |56                |embedding_dim
96                |64                |conv1d_1
32                |64                |fc_1
0.4               |0.2               |dropout_1
96                |48                |fc_2
0.2               |0.4               |dropout_2
4.2744e-06        |8.3161e-05        |learning_rate

Epoch 1/5
264/264 [==============================] - 6s 19ms/step - loss: 0.6917 - accuracy: 0.5329 - val_loss: 0.6832 - val_accuracy: 0.8084
Epoch 2/5
103/264 [==========>...................] - ETA: 2s - loss: 0.6839 - accuracy: 0.6775

KeyboardInterrupt: 

## Trainer
Setelah mendapatkan hyperparameter terbaik, melatih model menggunakan Trainer. Modul trainer berisi logika pelatihan model.

In [ ]:
TRAINER_MODULE_FILE = "modules/trainer.py"

In [ ]:
trainer = Trainer(
    module_file=os.path.abspath(TRAINER_MODULE_FILE),
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(splits=['train']),
    eval_args=trainer_pb2.EvalArgs(splits=['eval']),
    hyperparameters=tuner.outputs['best_hyperparameters']
)

In [ ]:
interactive_context.run(trainer)

{'space': [{'class_name': 'Int', 'config': {'name': 'vocab_size', 'default': None, 'conditions': [], 'min_value': 10000, 'max_value': 15000, 'step': 1000, 'sampling': 'linear'}}, {'class_name': 'Int', 'config': {'name': 'sequence_length', 'default': None, 'conditions': [], 'min_value': 75, 'max_value': 150, 'step': 25, 'sampling': 'linear'}}, {'class_name': 'Int', 'config': {'name': 'embedding_dim', 'default': None, 'conditions': [], 'min_value': 16, 'max_value': 128, 'step': 8, 'sampling': 'linear'}}, {'class_name': 'Int', 'config': {'name': 'conv1d_1', 'default': None, 'conditions': [], 'min_value': 32, 'max_value': 128, 'step': 16, 'sampling': 'linear'}}, {'class_name': 'Int', 'config': {'name': 'fc_1', 'default': None, 'conditions': [], 'min_value': 32, 'max_value': 256, 'step': 32, 'sampling': 'linear'}}, {'class_name': 'Float', 'config': {'name': 'dropout_1', 'default': 0.0, 'conditions': [], 'min_value': 0.0, 'max_value': 0.5, 'step': 0.1, 'sampling': 'linear'}}, {'class_name': 

INFO:tensorflow:Assets written to: outputs\hoax-detection\Trainer\model\90\Format-Serving\assets


INFO:tensorflow:Assets written to: outputs\hoax-detection\Trainer\model\90\Format-Serving\assets


527/527 [==============================] - 10s 17ms/step - loss: 0.3491 - accuracy: 0.8628 - val_loss: 0.5033 - val_accuracy: 0.8165 - lr: 1.6788e-04
Epoch 2/10
527/527 [==============================] - ETA: 0s - loss: 0.0134 - accuracy: 0.9987
Epoch 2: val_loss did not improve from 0.50329

Epoch 2: ReduceLROnPlateau reducing learning rate to 5.0364909111522134e-05.
527/527 [==============================] - 8s 14ms/step - loss: 0.0134 - accuracy: 0.9987 - val_loss: 0.7415 - val_accuracy: 0.8153 - lr: 1.6788e-04
Epoch 3/10
525/527 [============================>.] - ETA: 0s - loss: 0.0033 - accuracy: 0.9993
Epoch 3: val_loss did not improve from 0.50329

Epoch 3: ReduceLROnPlateau reducing learning rate to 1.510947295173537e-05.
527/527 [==============================] - 8s 15ms/step - loss: 0.0033 - accuracy: 0.9993 - val_loss: 0.7835 - val_accuracy: 0.8177 - lr: 5.0365e-05
Epoch 4/10
527/527 [==============================] - ETA: 0s - loss: 0.0027 - accuracy: 0.9993
Epoch 4: val_lo

INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:Assets written to: outputs\hoax-detection\Trainer\model\90\Format-Serving\assets


INFO:tensorflow:Assets written to: outputs\hoax-detection\Trainer\model\90\Format-Serving\assets


ExecutionResult(
    component_id: Trainer
    execution_id: 90
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={})
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}))

# Analisis dan Validasi Model

## Resolver
Menggunakan Resolver untuk mendapatkan model yang sudah diblessing atau diharapkan sebelumnya sebagai baseline untuk evaluasi model baru.

In [ ]:
model_resolver = Resolver(
    strategy_class= LatestBlessedModelStrategy,
    model = Channel(type=Model),
    model_blessing = Channel(type=ModelBlessing)
).with_id('Latest_blessed_model_resolver')
 
interactive_context.run(model_resolver)

ExecutionResult(
    component_id: Latest_blessed_model_resolver
    execution_id: 91
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Latest_blessed_model_resolver, output_key=model, additional_properties={}, additional_custom_properties={})
        model_blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Latest_blessed_model_resolver, output_key=model_blessing, additional_properties={}, additional_custom_properties={}))

## Evaluator
Pada tahap ini, evaluasi model yang baru dilatih menggunakan Evaluator. Evaluasi dilakukan dengan membandingkan model baru terhadap baseline model.

In [ ]:
eval_config = tfma.EvalConfig(
    model_specs=[tfma.ModelSpec(label_key='label')],
    slicing_specs=[tfma.SlicingSpec()],
    metrics_specs=[
        tfma.MetricsSpec(metrics=[
            tfma.MetricConfig(class_name='ExampleCount'),
            tfma.MetricConfig(class_name='AUC'),
            tfma.MetricConfig(class_name='FalsePositives'),
            tfma.MetricConfig(class_name='TruePositives'),
            tfma.MetricConfig(class_name='FalseNegatives'),
            tfma.MetricConfig(class_name='TrueNegatives'),
            tfma.MetricConfig(class_name='BinaryAccuracy',
                threshold=tfma.MetricThreshold(
                    value_threshold=tfma.GenericValueThreshold(
                        lower_bound={'value':0.5}),
                    change_threshold=tfma.GenericChangeThreshold(
                        direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                        absolute={'value':0.0001}
                        )
                    )
                )
            ]
        )
    ]
)

In [ ]:
evaluator = Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    baseline_model=model_resolver.outputs['model'],
    eval_config=eval_config)
 
interactive_context.run(evaluator)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


ExecutionResult(
    component_id: Evaluator
    execution_id: 92
    outputs:
        evaluation: OutputChannel(artifact_type=ModelEvaluation, producer_component_id=Evaluator, output_key=evaluation, additional_properties={}, additional_custom_properties={})
        blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Evaluator, output_key=blessing, additional_properties={}, additional_custom_properties={}))

In [ ]:
# Visualize the evaluation results
eval_result = evaluator.outputs['evaluation'].get()[0].uri
tfma_result = tfma.load_eval_result(eval_result)
tfma.view.render_slicing_metrics(tfma_result)
tfma.addons.fairness.view.widget_view.render_fairness_indicator(
    tfma_result
)

FairnessIndicatorViewer(slicingMetrics=[{'sliceValue': 'Overall', 'slice': 'Overall', 'metrics': {'accuracy': …

# Model Deployment

## Pusher
Terakhir, melakukan deployment model menggunakan Pusher. Model yang telah divalidasi dan diberkati akan dipush ke direktori penyimpanan yang telah ditentukan.

In [ ]:
# Tentukan path ke direktori tempat model akan dipush
serving_model_dir = 'serving_model_dir/hoax-detection-model'

# Tambahkan versi 1 pada direktori keluaran
push_destination = pusher_pb2.PushDestination(
    filesystem=pusher_pb2.PushDestination.Filesystem(
        base_directory=f'{serving_model_dir}/1'))

# Konfigurasi komponen Pusher
pusher = Pusher(
    model=trainer.outputs['model'],
    model_blessing=evaluator.outputs['blessing'],
    push_destination=push_destination
)

# Jalankan pusher dalam konteks interaktif
interactive_context.run(pusher)

ExecutionResult(
    component_id: Pusher
    execution_id: 93
    outputs:
        pushed_model: OutputChannel(artifact_type=PushedModel, producer_component_id=Pusher, output_key=pushed_model, additional_properties={}, additional_custom_properties={}))